In [ ]:
# Parameters
raw_dir = "raw"

In [ ]:
read_full_path = f"include/data/bronze/breweries/{raw_dir[0:8]}/{raw_dir}.json"

In [ ]:
rawPath = f"include/data/silver/breweries/{raw_dir[0:8]}"
fullPath= f"{rawPath}/{raw_dir}.parquet"

In [200]:
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder \
        .appName("Brewery Silver Layer") \
        .getOrCreate()

In [ ]:
if raw_dir == "raw":
    raw_path = f"../data/bronze/breweries/{datetime.now().strftime('%Y-%m-%d')}"
    read_full_path= f"{raw_path}/{datetime.now().strftime('%Y-%m-%dT%H_%M')}.json"
    if not os.path.exists(read_full_path):
        print(f"The file does not exists {read_full_path}")
        os.sys.exit(1)

In [ ]:
print(read_full_path)

../data/bronze/breweries/raw/2025-02-09T20_24.json


In [ ]:
schema = StructType([
    StructField("id", StringType(), False),
    StructField("name", StringType(), False),
    StructField("brewery_type", StringType(), False),
    StructField("address_1", StringType(), True),
    StructField("address_2", StringType(), True),
    StructField("address_3", StringType(), True),
    StructField("city", StringType(), False),
    StructField("state_province", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("country", StringType(), True),
    StructField("longitude", StringType(), False),
    StructField("latitude", StringType(), False),
    StructField("phone", StringType(), True),
    StructField("website_url", StringType(), True),
    StructField("state", StringType(), True),
    StructField("street", StringType(), True)
])

In [ ]:
df_json = spark.read.option("multiline", "true").json(read_full_path,schema)

+--------------------+--------------------+------------+--------------------+---------+---------+--------------+--------------+-----------+-------------+------------------+-----------------+------------+--------------------+-------------+--------------------+
|                  id|                name|brewery_type|           address_1|address_2|address_3|          city|state_province|postal_code|      country|         longitude|         latitude|       phone|         website_url|        state|              street|
+--------------------+--------------------+------------+--------------------+---------+---------+--------------+--------------+-----------+-------------+------------------+-----------------+------------+--------------------+-------------+--------------------+
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|      1716 Topeka St|     NULL|     NULL|        Norman|      Oklahoma| 73069-8224|United States|      -97.46818222|      35.25738891|  4058160490|http://www.405bre

In [ ]:
df_json.write.mode("overwrite").partitionBy("city").parquet(fullPath)

In [ ]:
spark.stop()

In [ ]:
print(f"[Silver] Finished Breweries in: {fullPath}")